## Creating a WebMap from the MapViewer Widget


### Set your own properties

In [1]:
gemeente = "Bronckhorst"

#### Connect to your ArcGIS Portal

In [2]:
from arcgis.gis import GIS
gis = GIS("home")

/opt/conda/lib/python3.7/site-packages/arcgis/gis/__init__.py:575: UserWarning:

You are logged on as maartje_dev_admin with an administrator role, proceed with caution.



#### Let's get some layers to add to the map

In [3]:
gemeentehuizenItem = gis.content.get("9b231e5a9d984d398f552eceaddaca21")
gemeentehuizenLayer = gemeentehuizenItem.layers[0]
print(f"Found layer: {gemeentehuizenLayer.properties.name}")

Found layer: Gemeentehuizen


In [4]:
gemeenteGrenzenItem = gis.content.get("96e45f8f0f304bfd84a8ab58edfacbd7")
gemeenteGrenzenLayer = gemeenteGrenzenItem.layers[0]
print(f"Found layer: {gemeenteGrenzenLayer.properties.name}")

Found layer: CBS_Gemeente


#### Create a new Map and open the MapViewer Widget

In [5]:
newMap = gis.map(f"{gemeente}, Netherlands")

#### Get the results from the Gemeenteraadverkiezingen 2022

In [6]:
detailsVerkiezingen = gis._con.get("https://www.verkiezingsuitslagen.nl/verkiezingen/detailJson/GR20220316")
presentGemeentes = {regio["Value"]: regio["Id"] for regio in detailsVerkiezingen["Regios"]["Regios"][0]["Options"]}

In [7]:
if gemeente in presentGemeentes:
    gemeenteId = presentGemeentes[gemeente]
    uitslagJson = gis._con.get(f"https://www.verkiezingsuitslagen.nl/verkiezingen/detailJson/GR20220316/{gemeenteId}")
    winner = uitslagJson["PieChart"][0]["Name"]
    colorHex = uitslagJson["PieChart"][0]["Color"]
    if not colorHex:
        color = [247,163,92]
    else:
        color = [int(colorHex[1:][i:i+2], 16) for i in (0, 2, 4)]
else:
    winner = "Nog niet bekend"
    color = [255, 255, 255]
print(f"The winner in Gemeente {gemeente} is {winner}")

The winner in Gemeente Bronckhorst is Gemeentebelangen Bronckhorst (GBB)


#### Add a graphic with the <i>Gemeentegrens</i> to the map, setting the color to the winner

In [8]:
newMap.clear_graphics()

selectedGemeente = gemeenteGrenzenLayer.query(f"statnaam = '{gemeente}'", out_fields=["statnaam"])
selectedGemeente.features[0].attributes["winner"] = f"De grootste partij is: {winner}"
newMap.draw(selectedGemeente, attributes={"winner": winner}, popup={"title": "statnaam", "content": "winner"}, symbol={
      "type": "esriSFS",
      "style": "esriSFSSolid",
      "color": color,
        "outline": {
         "type": "esriSLS",
         "style": "esriSLSSolid",
         "color": [110,110,110,100],
         "width": 1
        }
    })

#### Add the layer with the <i>Gemeentehuizen</i> to the map, using a Definition Expression

In [9]:
newMap.add_layer(gemeentehuizenLayer, {"definition_expression": f"gemeente = '{gemeente}'"})

#### View the result

In [10]:
display(newMap)

MapView(layout=Layout(height='400px', width='100%'))

#### Save the map as a new WebMap Item in your Content

In [13]:
webmapProperties = {
    "title": f"Gemeenteraadsverziekingen 2022: {gemeente}",
    "snippet": "WebMap created from MapViewer Widget in a Notebook",
    "tags": ["automation", "python", "MapViewer", "Notebook", gemeente]
}
webmapItem = newMap.save(webmapProperties)
display(webmapItem)

<Item title:"Gemeenteraadsverziekingen 2022: Bronckhorst" type:Web Map owner:maartje_dev_admin>